Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [2]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [4]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [5]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

In [6]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   33.6s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [7]:
grid_search.best_score_

0.8868144690781797

In [8]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1])

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [9]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
train.head()

trainX = train['description']
trainy = train['category']

In [10]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


### Define Pipeline Components

In [11]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [12]:
parameters = {
    'vect__max_df': ( 0.65, 1.0),
    'vect__min_df': (.01, .1),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10, 50),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(trainX, trainy)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    7.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   36.3s
[Parallel(n_jobs=4)]: Done 240 out of 240 | elapsed:   47.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'vect__max_df': (0.65, 1.0), 'vect__min_df': (0.01, 0.1), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10, 50), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [13]:
grid_search.best_score_

0.874323279195669

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [14]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [15]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [16]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [17]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission3.csv', index=False)

## Challenge

You're trying to achienve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [18]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [19]:
params = { 
    'lsi__svd__n_components': (10,100),
    'lsi__vect__max_df':(.9, .95, 1.0),
    'clf__n_estimators':(5,10),
}

In [20]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [21]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=10, n_jobs=-1, verbose=1)
grid_search.fit(trainX, trainy)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   40.1s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  4.0min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lsi__svd__n_components': (10, 100), 'lsi__vect__max_df': (0.9, 0.95, 1.0), 'clf__n_estimators': (5, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [22]:
grid_search.best_score_

0.8921113689095128

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [23]:
lsi = Pipeline([('vect', vect), ('svd', svd)])
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [24]:
parameters = {
    'lsi__svd__n_components': [10,100,250],
    'vect__max_df': [0.75, 1.0],
    'clf__max_depth':[5,10,15,20]
}

grid_search = GridSearchCV(pipe,parameters, cv=10, n_jobs=-1, verbose=1)
grid_search.fit(trainX, trainy)

Fitting 10 folds for each of 24 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


ValueError: Invalid parameter vect for estimator Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]). Check the list of available parameters with `estimator.get_params().keys()`.

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission4.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [111]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [116]:
doc = nlp("Two bananas in pyjamas")

In [125]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [126]:
X = get_word_vectors(data.data)

len(X) == len(data.data)

True

In [127]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [128]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/mph/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [129]:
rfc.score(X, data.target)

0.9859976662777129

## Follow Along

In [161]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [163]:
tokenize = Tokenizer(nlp.vocab)

In [171]:
[token.text.lower() for token in tokenize(train['description'][0])]

['a',
 'marriage',
 'of',
 '13',
 'and',
 '18',
 'year',
 'old',
 'bourbons.',
 'a',
 'mature',
 'yet',
 'very',
 'elegant',
 'whiskey,',
 'with',
 'a',
 'silky',
 'texture',
 'and',
 'so',
 'easy',
 'to',
 'embrace',
 'with',
 'a',
 'splash',
 'of',
 'water.',
 'balanced',
 'notes',
 'of',
 'honeyed',
 'vanilla,',
 'soft',
 'caramel,',
 'a',
 'basket',
 'of',
 'complex',
 'orchard',
 'fruit,',
 'blackberry,',
 'papaya,',
 'and',
 'a',
 'dusting',
 'of',
 'cocoa',
 'and',
 'nutmeg;',
 'smooth',
 'finish.',
 'sophisticated,',
 'stylish,',
 'with',
 'well-defined',
 'flavors.',
 'a',
 'classic!']

In [179]:
# Pipe

tokens = []

for doc in tokenize.pipe(train['description'], batch_size=500):
    doc_tokens = [token.text.lower() for token in doc]
    tokens.append(doc_tokens)
    
train['tokens'] = tokens

In [198]:
# X = get_word_vectors(train['description'])


In [184]:
# rfc.fit(X, train['category'])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [193]:
# rfc.score(X, train['category'])

0.9930394431554525

In [201]:
def get_lemmas(text):

    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

In [203]:
train['lemmas'] = train['description'].apply(get_lemmas)

In [206]:
train.head()

,id,description,category,tokens,lemmas
0,1,A marriage of 13 and 18 year old bourbons. A m...,2,"[a, marriage, of, 13, and, 18, year, old, bour...","[marriage, 13, 18, year, old, bourbon, mature,..."
1,2,There have been some legendary Bowmores from t...,1,"[there, have, been, some, legendary, bowmores,...","[legendary, Bowmores, mid-60, bit, equal, shar..."
2,3,This bottling celebrates master distiller Park...,2,"[this, bottling, celebrates, master, distiller...","[bottling, celebrate, master, distiller, Parke..."
3,4,What impresses me most is how this whisky evol...,1,"[what, impresses, me, most, is, how, this, whi...","[impress, whisky, evolve, incredibly, complex,..."
4,9,"A caramel-laden fruit bouquet, followed by une...",2,"[a, caramel-laden, fruit, bouquet,, followed, ...","[caramel, laden, fruit, bouquet, follow, unend..."


In [219]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tokenize(document):
    doc = nlp(document)
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.98)
dtm = tfidf.fit_transform(train['description'])

In [220]:
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [221]:
print(dtm.shape)
dtm.head()

(2586, 302)


,,10,100,12,add,age,aged,almond,amber,american,...,whisky,white,wine,wood,year,young,youthful,£,’,’s
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.134752,0.0,0.0,0.0,0.000000,0.00000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.159722,0.00000
2,0.134784,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.310059,0.0,0.0,0.0,0.211350,0.12388
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.346884,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000


In [262]:
from xgboost import XGBClassifier

In [271]:
lsi = Pipeline([('vect', vect), ('svd', svd)])
vect = TfidfVectorizer( stop_words='english', ngram_range=(1,2))
clf = XGBClassifier()

svd = TruncatedSVD(n_components=300, 
                   algorithm='randomized',
                   n_iter=20)

lsi = Pipeline([('vect', vect), ('svd', svd)])
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

parameters = {
    'lsi__svd__n_components': (10,55),
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth': (5,10,15),
    'clf__n_estimators': (200, 500, 1000),
    'lsi__vect__min_df': (0.025, 0.05)
    

}

grid_search = GridSearchCV(pipe, parameters, cv=10, n_jobs=-1, verbose=10)
grid_search.fit(trainX, trainy)

Fitting 10 folds for each of 72 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   12.0s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   12.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.4s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   47.9s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  69 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 129 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  7

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lsi__svd__n_components': (10, 55), 'lsi__vect__max_df': (0.75, 1.0), 'clf__max_depth': (5, 10, 15), 'clf__n_estimators': (200, 500, 1000), 'lsi__vect__min_df': (0.025, 0.05)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [272]:
grid_search.best_score_

0.9025522041763341

In [273]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [274]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [275]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [276]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission8.csv', index=False)

In [269]:
trainX.head()

0    A marriage of 13 and 18 year old bourbons. A m...
1    There have been some legendary Bowmores from t...
2    This bottling celebrates master distiller Park...
3    What impresses me most is how this whisky evol...
4    A caramel-laden fruit bouquet, followed by une...
Name: description, dtype: object

In [1]:
print(test.shape)
train.shape

NameError: name 'test' is not defined

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on